In [1]:
# Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

%run excute_nonZS_MaxMin.ipynb



In the context of graph neural networks (GNNs), the input data typically consists of three parts:

The graph structure: This is typically represented as an adjacency matrix or an edge list. The adjacency matrix is a square matrix where the (i,j)-th entry is 1 if there is an edge from node i to node j, and 0 otherwise. An edge list is simply a list of tuples (i,j) representing the edges in the graph.

Node features: These are the properties of each node in the graph that the model uses to learn representations. Node features can include things like text embeddings, image features, or numerical attributes. In the case of the Cora dataset, each node is represented by a 1433-dimensional binary vector indicating the presence or absence of certain keywords in a document.

Node labels: These are the ground truth labels for each node that the model tries to predict. In the case of the Cora dataset, each node is assigned one of seven possible labels indicating the topic of the corresponding document.
The data for the Cora dataset looks like this:

css
Copy code
Data(edge_index=[2, 10556], x=[2708, 1433], y=[2708])
Here, edge_index is the edge list representing the graph structure, x is the node feature matrix, and y is the node label vector.

In more detail, edge_index is a 2D tensor of shape (2, E) where E is the number of edges in the graph. 
The first row of edge_index contains the indices of the source nodes of each edge, while the second row contains the indices of the target nodes. 
Each row of x represents the feature vector for a node, and y is a 1D tensor of length N where N is the number of nodes in the graph. 
The i-th entry of y contains the label for the i-th node.

In [7]:
# # create the data
# adj_list = []
# feat_list = []
# labels_list = []
# for i in range(10):
#     (v1,v2, max_opinion, min_opinion,max_pol, s, G, a,b,c,d) = run(n)
#     adj_list.append(torch.tensor(G))
#     feat_list.append(torch.tensor(s))
#     #create min_action label
#     c1 = [0 for i in range(20)]
#     c1[v2]=1
#     print(c1)
#     labels= c1
#     labels_list.append(torch.tensor(labels))

In [2]:
# load the graph data from multiple files
graph_data = []
for i in range(num_graphs):
    adj = np.load(f'graph_{i}_adj.npy')
    node_features = np.load(f'graph_{i}_features.npy')
    labels = np.load(f'graph_{i}_labels.npy')
    graph_data.append((adj, node_features, labels))

# Convert each graph data into PyTorch Geometric format
data_list = []
for i, (adj, features, labels) in enumerate(graph_data):
    edge_index = torch.tensor(adj.nonzero(), dtype=torch.long).t()
    x = torch.tensor(features, dtype=torch.float)
    y = torch.tensor(labels, dtype=torch.long)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    data_list.append(data)

# Create a PyTorch Geometric DataLoader to handle batching and shuffling
loader = DataLoader(data_list, batch_size=batch_size, shuffle=True)

NameError: name 'num_graphs' is not defined

In [ ]:
#Graphical Convelution Network model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)  # define the first GCN layer
        self.conv2 = GCNConv(hidden_dim, output_dim) # define the second GCN layer

    def forward(self, data):
        # Pass the node features and adjacency matrix through the first layer
        x, edge_index = data.x, data.edge_index  # read each variable
        x = F.relu(self.conv1(x, edge_index)) # apply the ReLU activation function on first layer
        # Pass the output of the first layer through the second layer
        x = self.conv2(x, edge_index)  
        return F.log_softmax(x, dim=1)

In [ ]:
# Prepare the data
adjacency_matrix = [[0, 1, 1, 0], [1, 0, 1, 1], [1, 1, 0, 1], [0, 1, 1, 0]]
num_nodes = len(adjacency_matrix)
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if adjacency_matrix[i][j] == 1],
                          dtype=torch.long).t()
node_features = torch.randn((num_nodes, 16))
node_labels = torch.tensor([0, 1, 1, 0], dtype=torch.long)

data = Data(x=node_features, edge_index=edge_index, y=node_labels)

In [ ]:
# Prepare the data
# Create a torch geometric data object
# data = Data(x=torch.tensor(node_props, dtype=torch.float),
#             edge_index=torch.tensor(adj_matrix.nonzero(), dtype=torch.long),
#             y=torch.tensor(labels, dtype=torch.long))

# Split the data into training and validation sets
train_data, val_data = torch.utils.data.random_split(data, [int(0.8*len(data)), len(data)-int(0.8*len(data))])

# Create data loaders for batch processing
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [ ]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(input_dim=node_props.shape[1], hidden_dim=64, output_dim=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

def train(model, loader, optimizer, criterion, device):
    model.train()   # set the model in training mode
    total_loss, total_correct = 0, 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()  # zero the gradients
        
        # forward pass
        output = model(data)  
        loss = criterion(output, data.y)  # compute the loss on labeled nodes only
        
        # backward pass
        loss.backward() 
        optimizer.step()  # update the parameters
        total_loss += loss.item()
        total_correct += (output.argmax(dim=1) == data.y).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

def test(model, loader, criterion, device):
    model.eval() # set the model in evaluation mode
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)  # forward pass
            loss = criterion(output, data.y) 
            total_loss += loss.item()
            total_correct += (output.argmax(dim=1) == data.y).sum().item() # compute the number of correct predictions
    return total_loss/len    # compute the accuracy

In [ ]:
import matplotlib.pyplot as plt

train_losses = []
test_losses = []
start = time.process_time()

for epoch in range(200):
    train_loss = train() # train the model for one epoch
    test_losses = test() # evaluate the model on the test set
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    if e % 5 == 0:
        print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
            e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

plt.figure(figsize=(12,8))
plt.plot(train_losses, label='Train loss', c='blue')
plt.plot(test_losses, label='Test loss', c='red')
plt.xlabel('Epochs')
plt.ylabel('Accurarcy')
plt.title('GCN')
plt.legend(loc='lower right', fontsize='x-large')
plt.savefig('gcn_loss.png')
plt.show()

In [ ]:
! pip uninstall ipykernel
! pip uninstall ipython
! pip uninstall jupyter_client
! pip uninstall jupyter_core
! pip uninstall traitlets
! pip uninstall python_genutils

^C
^C


In [ ]:
! conda clean -tipsy

In [ ]:

! pip install ipykernel
! pip install ipython
! pip install jupyter_client
! pip install jupyter_core
! pip install traitlets
! pip install python_genutils